In [1]:
import numpy as np
from numpy import array
from math import sqrt
from numpy import zeros

In [3]:
train = np.loadtxt("train.csv",delimiter=',')
test = np.loadtxt("test.csv",delimiter=',')
passive = np.loadtxt("passive.csv",delimiter=',')

In [4]:
print(train.shape)
print(test.shape)
print(passive.shape)

(51, 1683)
(51, 1683)
(448, 1683)


In [5]:
def matr(shp):
    temp = zeros(shp)
    temp[0,:] = range(shp[1])
    temp[:,0] = range(shp[0])
    return temp

def simlarity(row1,passive_user,k):
    k_neighbr,idx,arr = np.zeros(passive_user.shape[0]),0,passive[:,0]
#     print passive_user.shape[0]
    for row2 in passive_user[:,1:]:
        temp1,temp2,temp3 = 0,0,0
#         print np.sum([row1], axis=1)
        mu1 = np.sum([row1], axis=1)[0]/row1.nonzero()[0].shape[0]
        mu2 = np.sum([row2], axis=1)[0]/row2.nonzero()[0].shape[0]
        for i in range(len(row1)):
            if row1[i] is not 0 and row2[i] is not 0:
                temp1 += ((row1[i] - mu1) * (row2[i] - mu2))
                temp2 += (row1[i] - mu1) * (row1[i] - mu1)
                temp3 += (row2[i] - mu2) * (row2[i] - mu2)
        k_neighbr[idx] = temp1/(sqrt(temp2) * sqrt(temp3))
        idx+=1
    return arr[k_neighbr.argsort()][-k:][::-1]

def k_neighbor(data,passive_user,k=30):
    users,idx = matr((data.shape[0],k+1)),1
    for user in data[1:,1:]:
        users[idx,1:],users[idx,0] = simlarity(user,passive[1:],k),data[idx,0]
        idx += 1    
    return users

In [6]:
relation_matrix = k_neighbor(train,passive)

In [7]:
# a = array([5, 2, 7, 4, 4, 2, 8, 6, 4, 4])
# a[::-1].sort()
# print a.argsort()[-3:][::-1]
# print a.argsort()[-3:][::-1] + [1,1,1]
# print a[:3]
# print a.argsort()[-3:]
# print a[a.argsort()[-3:][::-1]]
relation_matrix.shape
relation_matrix[1,:]

array([ 536.,  507.,  147.,  696.,  745.,  341.,  290.,  636.,  328.,
        621.,   47.,  450.,  386.,  926.,  320.,  912.,  425.,  114.,
        393.,  876.,   98.,  471.,  478.,   74.,   81.,  452.,   61.,
        805.,  917.,  238.,  342.])

In [7]:
np.savetxt("relation_martix.csv",relation_matrix,delimiter=',')

# Predictions

In [10]:
def prediction(test,relation_matrix,passive):
    idx,idx_passive = 0,{}
    for i in passive[:,0]:
        idx_passive[i] = idx
        idx += 1
    sim_users = zeros(relation_matrix.shape[1]-1)
    idx,error =  1,zeros(test.shape[0])
#     print error.shape
    for pred_movie,rel_user in zip(test[1:,:],relation_matrix[1:,1:]): 

        sim_users = array([idx_passive[x] for x in rel_user])
        test_nonzero = pred_movie.nonzero()[0][1:]
        
        temp = passive[sim_users[:,None],test_nonzero]
        temp2 = np.sum(temp,axis=0)
        temp3 = temp2.nonzero()[0]
        temp4 = pred_movie[test_nonzero]
        temp5 = temp[:,temp3]
#         temp = temp.transpose()
#         temp = temp[~np.all(temp == 0, axis=1)].transpose()
        predicted_rating = np.average(temp5, axis=0, weights=temp5.astype(bool))
#         print pred_movie[test_nonzero]
#         print predicted_rating
#         print temp.sum(0)
#         print (temp != 0).sum(0)
#         predicted_rating = temp.sum(0) / (temp != 0).sum(0)
#         error[idx] = sqrt(((predicted_rating - temp4[temp3]) ** 2).mean(axis=0))
        error[idx] = (np.absolute(predicted_rating - temp4[temp3])).mean(axis=0)
        idx += 1
#     return error[1:]
#         temp1,temp2 = temp.sum(axis=0),temp.nonzero()[1].shape[0]
#         predicted_rating = temp1/temp2
#         print temp1
#         print temp2
#         print predicted_rating
#         print pred_movie[test_nonzero]
        
#         print (test_nonzero - predicted_rating)
#         print ((test_nonzero - predicted_rating)**2)/predicted_rating.shape[0]
#         error[idx] = (((test_nonzero - predicted_rating)**2)/predicted_rating.shape[0]).mean()
#         idx += 1
#         if idx is 3:
#             break
    return error[1:]

In [12]:
# d = prediction(test,relation_matrix,passive)
err = prediction(test,relation_matrix,passive)
err.mean()
# print test[1,1:].nonzero()[0]
# print "---------------------------------------------------"
# print test[1,1:]

0.87432955684993485

In [13]:
# test[1].nonzero()[0][1:]
# print d.keys()
# array([d[x] for x in relation_matrix[1,1:]])
# print relation_matrix[1,1:]
# print relation_matrix[2]
# print 181 in d.keys(ruby123
err
# a = array([1.,2,0,4])
# b = array([2,2,2,2])
# a/b
# a.astype(bool)
# print 181 in passive[:,0]
# print train[:,0]
# print passive[:,0]

array([ 0.81701999,  0.82374847,  0.85048048,  1.13475275,  0.85382937,
        1.37954879,  0.7724359 ,  0.77333333,  0.5275026 ,  0.79013862,
        0.97134921,  0.9272403 ,  0.82939796,  0.51611111,  1.24097207,
        0.59166667,  0.75096877,  1.10304579,  1.01212263,  0.77606872,
        0.53139083,  0.60117521,  0.59578969,  0.73775215,  0.99738595,
        0.82663626,  0.84374625,  0.7567487 ,  1.17373278,  0.9019721 ,
        0.56862745,  0.84839492,  0.81882716,  0.75321675,  0.99989732,
        1.0164966 ,  1.57400932,  0.75459296,  1.27418155,  0.51617133,
        1.77380952,  1.15123457,  0.85477559,  0.38110119,  0.95748796,
        0.85994018,  1.19848485,  0.79317918,  0.65511151,  0.55887446])

In [364]:
# a = array([[1.0,2,3,4],[8,7,9,0],[4,5,0,7],[4,2,2,4])
a = array([[1.0,2,3,0],[8,7,9,0],[4,5,0,0],[4,2,2,0]])
print a
# b = a[array([0,1,3])[:,None],array([2,3])]
# print b
# c = b.sum(axis=0)
# d = b.nonzero()[1]
# print c
# print d
# a = array([0,1,2,3,4,5,6,7,8,9])
# a.mean()
# a = a.transpose()
# b = ~np.all(a == 0, axis=1)
# a = a[~np.all(a == 0, axis=1)].transpose()
# print a[~np.all(a == 0, axis=0)]
# a = a.transpose()
a[:,b]

[[ 1.  2.  3.  0.]
 [ 8.  7.  9.  0.]
 [ 4.  5.  0.  0.]
 [ 4.  2.  2.  0.]]


array([[ 1.,  2.,  3.],
       [ 8.,  7.,  9.],
       [ 4.,  5.,  0.],
       [ 4.,  2.,  2.]])

In [349]:
# c/d
a

array([[ 1.,  2.,  3.],
       [ 8.,  7.,  9.],
       [ 4.,  5.,  0.],
       [ 4.,  2.,  2.]])

In [155]:
a[0,None]

array([[1, 2, 3, 4]])

In [171]:
a**2

array([[  1.,   4.,   9.,  16.],
       [ 64.,  49.,  81.,   0.],
       [ 16.,  25.,   0.,  49.],
       [ 16.,   4.,   4.,  16.]])

In [159]:
print relation_matrix[:,0]
print test[:,0]

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.]
[   0.  536.  310.  233.  255.  525.  863.  245.  845.  360.  499.  101.
   82.  453.  304.  180.  888.  879.  109.  335.   20.  558.  594.  552.
  497.  850.   76.  600.  338.  223.  232.  641.  221.  187.   64.  644.
    4.  127.  456.  475.   84.  681.  289.  275.  941.  541.  898.  353.
  795.  429.  652.]
